<a href="https://colab.research.google.com/github/josemoti1999/pulmonary_embolism_detection/blob/main/3_rsna_pulmonary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Experiment4 - with B0
* Experiment5 - with B7 - no label smoothing
* Experiment6 - with b5 - no label smoothing
* Experiment7 - b5 and imageposition - create new tfrecords with image position
* Experiment12 - with image position
* Experiment 13 - with efnetb7

In [ ]:
%tensorflow_version 2.x
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
colab = 1
FOLDER = 'Experiment_13'

In [ ]:
!pip install -q efficientnet
!pip install -q gcsfs
import os
import re, gc, time
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
from tqdm.notebook import tqdm as tqdm
from collections import namedtuple

In [ ]:
!pip install -q tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0
import requests
resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
if resp.status_code != 200:
  print("Failed to switch the TPU to TF {}".format(version))

In [ ]:
PATH = '/content/drive/My Drive/Kaggle/rsna_pulmonary'
PATH = os.path.join(PATH, FOLDER)
os.makedirs(PATH, exist_ok=True)
PATH

'/content/drive/My Drive/Kaggle/rsna_pulmonary/Experiment_13'

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

Running on TPU  grpc://10.89.49.106:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.89.49.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.89.49.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
BATCH_SIZE_PER_REPLICA = 32
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
IMAGE_SIZE = [256, 256]
SEED = 100

In [ ]:
GCS_PATHS = [
'gs://kds-675a4d0ca17253f876424309f74145c1167e831dc34c32522a29e4a9',
'gs://kds-1077ea6ef5bd8d0ba8150b786cee31c1c897708727eeb60a5fc2c0cd',
'gs://kds-1bc7664eb4ada6b80cb8c57e621c6cb78f2092fb164aea1302bd77a3',
'gs://kds-19a534b62267f9fe27cf1640e8781e6c04fd661d6b67ac160f1cc746',
'gs://kds-b5a1644b1f9084284cb2f724b41a5501d7fa4c9e5c4eb99a5a8d602f',
'gs://kds-f928be364facd6fe31db68406c543692388b880e8c7585f4c0403573',
'gs://kds-1a521e2e4a90df381780b48a4ce28e63764e7bec29805c8d6d6861e2',
'gs://kds-407358bc70721c44703d11afe83bdf2e970449eb110f376d01223862',
'gs://kds-db1695bd356de02fb8b89de44067a54e1db12f99675678d5e92b5917',
'gs://kds-2d74bd2971f4de2c28e3a427d05381f967de5070458b20031273e0f8',
'gs://kds-80c22cf93ec97373e7bbdc091dc2f9f7358e4c08d85797cdf86d9723',
'gs://kds-981f9b6ce021f08600630510b5398dafaca9ef15de304a20f083c886',
'gs://kds-54fafeca23b3db6855d075ce547486a53f37f981e0100552e714f561',
'gs://kds-9c42ec420192f5d63b360164ad37ed81dbfe7afc16a4e4fac418e952',
'gs://kds-1bc45eb2a6dcbb73ace5842a7b5c01effd9a3230f9b8c73314a26d74'
]

DICT_PATH = 'gs://kds-baad38a91ebb50744b66774d8ba77e8d3df2ce699dc5f8ad2d5dc6f6/train_with_metadata.csv'

In [ ]:
FILENAMES=[]
for GCS_PATH in GCS_PATHS:     
    FILENAMES += tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.02, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
print(training_groups, validation_groups)

df = pd.read_csv(DICT_PATH).set_index('SOPInstanceUID')
def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    n = df[df['fold'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')

[1, 12, 13, 6, 10, 5, 2, 4, 0, 11, 7, 3, 14, 8] [9]
Training with 1671055 images
Validating with 119539 images


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0
    #image = tf.image.random_flip_left_right(image)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example, return_id):
    TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64),
        "id" : tf.io.FixedLenFeature([], tf.string),
        "image_position" : tf.io.FixedLenFeature([], tf.float32),
        "q_i" : tf.io.FixedLenFeature([], tf.float32)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    target = tf.cast(tf.expand_dims(example['target'],axis=0), tf.int32)
    image_id = example['id']
    image_position = tf.cast(example['image_position'], tf.float32)
    q_i = tf.cast(example['q_i'], tf.float32)
    if return_id:
        return (image,image_position), q_i, image_id
    return (image,image_position), q_i, target

def load_dataset(filenames, ordered, return_id=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False   
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda example:read_tfrecord(example, return_id=return_id), num_parallel_calls = AUTO) 
    return dataset

def get_training_dataset(filenames, ordered):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(filenames, ordered):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_prediction_dataset(filenames, ordered, return_id):
    dataset = load_dataset(filenames, ordered=ordered, return_id=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

In [ ]:
@tf.function
def lrfn(epoch):
    LR_START = 0.000001
    LR_MAX = 0.000005 * strategy.num_replicas_in_sync
    LR_MIN = 0.000001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 5
    LR_EXP_DECAY = .8
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

In [ ]:
EPOCHS = 5
EFNET = 7
RETRAIN = False
CUSTOM_LR = True
LR = 1e-4
MODEL_PATH = '.h5'

In [ ]:
def int_div_round_up(a, b):
    return (a + b - 1) // b

def calc_loss(true, pred, q_i):
    binary_ce_loss = tf.keras.losses.binary_crossentropy(true, pred)
    sample_weight = tf.cast(q_i+1e-7,tf.float32)
    loss = binary_ce_loss*sample_weight
    return loss

class LRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __call__(self, step):
        return lrfn(epoch=step//STEPS_PER_EPOCH)

seed_everything(SEED)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = int_div_round_up(NUM_VALIDATION_IMAGES, BATCH_SIZE)

EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]
with strategy.scope():
    inp1 = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    inp2 = tf.keras.layers.Input(shape = (1), name = 'inp2')
    x1 = EFNS[EFNET](weights = 'noisy-student', include_top = False)(inp1)
    x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
    #x2 = tf.keras.layers.Dense(128,activation='relu')(inp2)
    #x = tf.keras.layers.concatenate((x1,x2), axis=-1)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(x1)
    model = tf.keras.models.Model(inputs = (inp1,inp2), outputs = output)

    if CUSTOM_LR:
        #optimizer = tf.keras.optimizers.Adam(learning_rate=LRSchedule())
        optimizer = tf.keras.optimizers.SGD(learning_rate=LRSchedule(), momentum=0.99)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=LR, momentum=0.99)
    
    train_auc = tf.keras.metrics.AUC()
    val_auc = tf.keras.metrics.AUC()
    train_loss = tf.keras.metrics.Sum()
    val_loss = tf.keras.metrics.Sum()
    train_q_i = tf.keras.metrics.Sum()
    val_q_i = tf.keras.metrics.Sum()
    loss_fn = lambda true,pred,q_i: tf.nn.compute_average_loss(calc_loss(true,pred,q_i), 
                                                     global_batch_size=1)
    q_i_sum_fn = lambda q_i: tf.nn.compute_average_loss(q_i, global_batch_size=1)

if RETRAIN:
    print('Loading model...')
    model.load_weights(os.path.join(PATH, MODEL_PATH))

258072576/258068648 [==============================] - 3s 0us/step


In [ ]:
print('Loading train data...')
train_dataset = strategy.experimental_distribute_dataset(get_training_dataset(TRAINING_FILENAMES, ordered = False))
print('Loading val data...')
val_dataset = strategy.experimental_distribute_dataset(get_validation_dataset(VALIDATION_FILENAMES, ordered = True))
print("Steps per epoch:", STEPS_PER_EPOCH)
print('Validation steps:', VALIDATION_STEPS)
History = namedtuple('History', 'history')
history = History(history={'loss': [], 'val_loss': [], 'auc': [], 'val_auc': []})

@tf.function
def train_step(x, q_i, target):
    with tf.GradientTape() as tape:
        probabilities = model(x, training=True)
        loss = loss_fn(target, probabilities, q_i)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_auc.update_state(target, probabilities)
    train_loss.update_state(loss)
    q_i_sum = q_i_sum_fn(q_i)
    train_q_i.update_state(q_i_sum)

@tf.function
def valid_step(x, q_i, target):
    probabilities = model(x, training=False)
    loss = loss_fn(target, probabilities, q_i)
    val_auc.update_state(target, probabilities)
    val_loss.update_state(loss)
    q_i_sum = q_i_sum_fn(q_i)
    val_q_i.update_state(q_i_sum)

Loading train data...
Loading val data...
Steps per epoch: 6527
Validation steps: 467


In [ ]:
start_time = epoch_start_time = time.time()
epoch = 0
for step, (x, q_i, target) in enumerate(train_dataset):

    strategy.run(train_step, args=(x, q_i, target))
    if step%2000==0:
        print(step,'--> ''Loss:',train_loss.result().numpy()/train_q_i.result().numpy(),'...','AUC:', train_auc.result().numpy())

    if ((step+1) // STEPS_PER_EPOCH) > epoch:
        print('|')
        for x, q_i, target in val_dataset:
            strategy.run(valid_step, args=(x, q_i, target))
        history.history['auc'].append(train_auc.result().numpy())
        history.history['val_auc'].append(val_auc.result().numpy())
        history.history['loss'].append(train_loss.result().numpy()/train_q_i.result().numpy())
        history.history['val_loss'].append(val_loss.result().numpy()/val_q_i.result().numpy())

        epoch_time = time.time() - epoch_start_time
        print('EPOCH {:d}/{:d}'.format(epoch+1, EPOCHS))
        print('time: {:0.1f}s'.format(epoch_time),
              'loss: {:0.4f}'.format(history.history['loss'][-1]),
              'auc: {:0.4f}'.format(history.history['auc'][-1]),
              'val_loss: {:0.4f}'.format(history.history['val_loss'][-1]),
              'val_auc: {:0.4f}'.format(history.history['val_auc'][-1]),
              'lr: {:0.4g}'.format(lrfn(epoch) if CUSTOM_LR else LR), flush=True)
        epoch = (step+1) // STEPS_PER_EPOCH
        epoch_start_time = time.time()
        train_auc.reset_states()
        val_auc.reset_states()
        val_loss.reset_states()
        train_loss.reset_states()
        train_q_i.reset_states()
        val_q_i.reset_states()

        NUM_INFERENCE_IMAGES  = count_data_items(VALIDATION_FILENAMES)
        print(f'Inference with {NUM_INFERENCE_IMAGES} images')
        check_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True)
        VALIDATION_STEPS = (NUM_INFERENCE_IMAGES // BATCH_SIZE) +1
        print('Predicting')
        pred = model.predict(check_dataset,steps=VALIDATION_STEPS,verbose=1)
        print('Pred shape',pred.shape)
        test_dataset = get_prediction_dataset(VALIDATION_FILENAMES, ordered = True, return_id=True)
        decoder = lambda x: x.decode("utf-8")
        count = 0
        for img, q_i, img_name in iter(test_dataset):
            img_name=img_name.numpy()
            img_name = np.vectorize(decoder)(img_name)
            if count==0:
                final_array = img_name
                count+=1
            else:
                final_array = np.concatenate((final_array, img_name), axis=0)
                count+=1
        val_df = pd.DataFrame({'SOPInstanceUID':final_array,'prediction':np.squeeze(pred,axis=-1)})
        val_df = pd.merge(val_df, df, on='SOPInstanceUID', how='left')
        def calc_image_level_loss(y_true_imag, y_pred_imag, q_image):
            score_i =  np.sum(- q_image * (y_true_imag*np.log(y_pred_imag) + (1-y_true_imag)*np.log(1-y_pred_imag))) / np.sum(q_image)
            print("{}: {:.6f}".format(("image_level_weighted_log_loss" +" "*50)[:30], score_i))
        calc_image_level_loss(val_df.pe_present_on_image.values, val_df.prediction.values+np.finfo(float).eps, val_df.q_i.values)
        
        model.save(os.path.join(PATH,'epoch-{}_loss-{:0.4f}_val_loss-{:0.4f}.h5'.format(epoch, history.history['loss'][-1],history.history['val_loss'][-1])))
        print('\n')

        if epoch >= EPOCHS:
            break

0 --> Loss: 0.70485365 ... AUC: 0.52952754
2000 --> Loss: 0.60824543 ... AUC: 0.59188527
4000 --> Loss: 0.55309546 ... AUC: 0.67680615
6000 --> Loss: 0.50756764 ... AUC: 0.72962826
|
EPOCH 1/5
time: 4955.1s loss: 0.4980 auc: 0.7386 val_loss: 0.3960 val_auc: 0.8165 lr: 1e-06
Inference with 119539 images
Predicting
467/467 [==============================] - 96s 206ms/step
Pred shape (119539, 1)
image_level_weighted_log_loss : 0.396010


8000 --> Loss: 0.38649687 ... AUC: 0.81503505
10000 --> Loss: 0.3665863 ... AUC: 0.8233856
12000 --> Loss: 0.35745528 ... AUC: 0.82886595
|
EPOCH 2/5
time: 4864.8s loss: 0.3518 auc: 0.8312 val_loss: 0.3199 val_auc: 0.8509 lr: 8.8e-06
Inference with 119539 images
Predicting
467/467 [==============================] - 89s 191ms/step
Pred shape (119539, 1)
image_level_weighted_log_loss : 0.319933


14000 --> Loss: 0.3425019 ... AUC: 0.8429548
16000 --> Loss: 0.3281689 ... AUC: 0.8464255
18000 --> Loss: 0.3237186 ... AUC: 0.85215294
|
EPOCH 3/5
time: 4844.7s l

# INFERENCE VAL DATA

In [ ]:
def get_model(ef):
    with strategy.scope():
        inp1 = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        inp2 = tf.keras.layers.Input(shape = (1), name = 'inp2')
        x1 = EFNS[EFNET](weights = 'noisy-student', include_top = False)(inp1)
        x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
        #x2 = tf.keras.layers.Dense(128,activation='relu')(inp2)
        #x = tf.keras.layers.concatenate((x1,x2), axis=-1)
        output = tf.keras.layers.Dense(1,activation='sigmoid')(x1)
        model = tf.keras.models.Model(inputs = (inp1,inp2), outputs = output)
        return model

EFNET = 5
NEW_MODEL = True
MODEL_PATH = 'epoch-2_loss-0.2604_val_loss-0.2779.h5'
if NEW_MODEL:
    model = get_model(ef=EFNET)
    print('Loading model...')
    model.load_weights(os.path.join(PATH, MODEL_PATH))
#with strategy.scope():
    #embedding = model.layers[-2].output
    #model = tf.keras.models.Model(inputs=model.input, outputs=embedding)

Loading model...


In [ ]:
NUM_INFERENCE_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Inference with {NUM_INFERENCE_IMAGES} images')
check_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True)
VALIDATION_STEPS = (NUM_INFERENCE_IMAGES // BATCH_SIZE) +1

print('Predicting')
pred = model.predict(check_dataset,steps=VALIDATION_STEPS,verbose=1)

print('pred min/max ',pred.min(), pred.max())
print('pred shape',pred.shape)
test_dataset = get_prediction_dataset(VALIDATION_FILENAMES, ordered = True, return_id=True)
decoder = lambda x: x.decode("utf-8")
count = 0
for img, q_i, img_name in iter(test_dataset):
    img_name=img_name.numpy()
    img_name = np.vectorize(decoder)(img_name)
    if count==0:
        final_array = img_name
        count+=1
    else:
        final_array = np.concatenate((final_array, img_name), axis=0)
        count+=1
print('final array shape', final_array.shape)

val_df = pd.DataFrame({'SOPInstanceUID':final_array,'prediction':np.squeeze(pred,axis=-1)})
val_df = pd.merge(val_df, df, on='SOPInstanceUID', how='left')

def calc_image_level_loss(y_true_imag, y_pred_imag, q_image):
    score_i =  np.sum(- q_image * (y_true_imag*np.log(y_pred_imag) + (1-y_true_imag)*np.log(1-y_pred_imag))) / np.sum(q_image)
    print("{}: {:.6f}".format(("image_level_weighted_log_loss" +" "*50)[:30], score_i))

calc_image_level_loss(val_df.pe_present_on_image.values, val_df.prediction.values+np.finfo(float).eps, val_df.q_i.values)

dict_ = dict(zip(final_array, pred))
with open(os.path.join(PATH, f'mapping_validation_set.p'), 'wb') as fp:
    pickle.dump(dict_, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(PATH, f'mapping_validation_set.p'), 'rb') as fp:
    data = pickle.load(fp)

Inference with 119539 images
Predicting
467/467 [==============================] - 127s 271ms/step
pred min/max  0.0 0.99983114
pred shape (119539, 1)
final array shape (119539,)
image_level_weighted_log_loss : 0.278059


# INFERENCE TRAIN DATA

In [ ]:
NUM_INFERENCE_IMAGES  = count_data_items(TRAINING_FILENAMES)
print(f'Inference with {NUM_INFERENCE_IMAGES} images')
val_dataset = get_validation_dataset(TRAINING_FILENAMES, ordered = True)
STEPS_PER_EPOCH = (NUM_INFERENCE_IMAGES // BATCH_SIZE) +1

print('Predicting')
pred = model.predict(val_dataset,steps=STEPS_PER_EPOCH,verbose=1)

print('pred min/max ',pred.min(), pred.max())
print('pred shape',pred.shape)
test_dataset = get_prediction_dataset(TRAINING_FILENAMES, ordered = True, return_id=True)
decoder = lambda x: x.decode("utf-8")
count = 0
for img, q_i, img_name in iter(test_dataset):
    img_name=img_name.numpy()
    img_name = np.vectorize(decoder)(img_name)
    if count==0:
        final_array = img_name
        count+=1
    else:
        final_array = np.concatenate((final_array, img_name), axis=0)
        count+=1
print('final array shape', final_array.shape)

val_df = pd.DataFrame({'SOPInstanceUID':final_array,'prediction':np.squeeze(pred,axis=-1)})
val_df = pd.merge(val_df, df, on='SOPInstanceUID', how='left')

def calc_image_level_loss(y_true_imag, y_pred_imag, q_image):
    score_i =  np.sum(- q_image * (y_true_imag*np.log(y_pred_imag) + (1-y_true_imag)*np.log(1-y_pred_imag))) / np.sum(q_image)
    print("{}: {:.6f}".format(("image_level_weighted_log_loss" +" "*50)[:30], score_i))

calc_image_level_loss(val_df.pe_present_on_image.values, val_df.prediction.values+np.finfo(float).eps, val_df.q_i.values)

dict_ = dict(zip(final_array, pred))
with open(os.path.join(PATH, f'mapping_train_set.p'), 'wb') as fp:
    pickle.dump(dict_, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(PATH, f'mapping_train_set.p'), 'rb') as fp:
    data = pickle.load(fp)

Inference with 1671055 images
Predicting
6528/6528 [==============================] - 1550s 237ms/step
pred min/max  0.0 0.9999092
pred shape (1671055, 1)
final array shape (1671055,)
image_level_weighted_log_loss : 0.208156
